In [1]:
#%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.metrics import log_loss 
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import warnings
warnings.simplefilter(action='ignore')

In [2]:
test = pd.read_csv('test_listo.csv')
train = pd.read_csv('train_listo.csv')

In [3]:
 df = pd.concat([train,test])

* One Hot Encoding y Label Encoding

In [5]:
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

In [6]:
def LabelEncord_categorical(df):
    categorical_features = ['Region','Territory','Source','Billing_Country', 'Account_Name','Account_Owner',\
                            'Opportunity_Owner','Account_Type', 'Opportunity_Type', 'Quote_Type','Delivery_Quarter', \
                            'Delivery_Terms','Last_Modified_By',\
                            'Product_Family', 'Product_Name','ASP_Currency','Prod_Category_A', 'Hemisferio','Estacin']
    for features in categorical_features:
        le = LabelEncoder()
        df[features] = le.fit_transform(df[features].astype(str))
    return df

def dummies(df):
    categorical_features = []
    for features in categorical_features:
        dummies =  pd.get_dummies(df[features])
        df = pd.concat([df, dummies],axis=1)
    return df

def pre_processing(df):
    df = LabelEncord_categorical(df)
    return df

df = pre_processing(df)
#train = pre_processing(train)
#test = pre_processing(test)

In [7]:
train =train[['ID']]
train = pd.merge(train,df, on= 'ID', how = 'inner') 
test =test[['ID']]
test = pd.merge(test,df, on= 'ID', how = 'inner') 
train.loc[train['Stage']=='Closed Won','Stage']=1
train.loc[train['Stage']=='Closed Lost','Stage']=0
train.Stage=train.Stage.astype('float')
train.to_csv("train_label_enc.csv", index=False)
test.to_csv("test_label_enc.csv", index=False)

* Target Encoding

In [220]:
df['Brand'] = df['Brand'].astype('category')
df['Territory'] = df['Territory'].astype('category')
df['Region'] = df['Region'].astype('category')
df['Bureaucratic_Code'] = df['Bureaucratic_Code'].astype('category')
df['Billing_Country'] = df['Billing_Country'].astype('category')
df['Source'] = df['Source'].astype('category')
df['Account_Name'] = df['Account_Name'].astype('category')
df['Account_Owner'] = df['Account_Owner'].astype('category')
df['Opportunity_Owner'] = df['Opportunity_Owner'].astype('category')
df['Account_Type'] = df['Account_Type'].astype('category')
df['Opportunity_Type'] = df['Opportunity_Type'].astype('category')
df['Quote_Type'] = df['Quote_Type'].astype('category')
df['Delivery_Quarter'] = df['Delivery_Quarter'].astype('category')
df['Delivery_Terms'] = df['Delivery_Terms'].astype('category')
df['Last_Modified_By'] = df['Last_Modified_By'].astype('category')
df['Product_Family'] = df['Product_Family'].astype('category')
df['Product_Name'] = df['Product_Name'].astype('category')
df['ASP_Currency'] = df['ASP_Currency'].astype('category')
df['Prod_Category_A'] = df['Prod_Category_A'].astype('category')
df['Product_Category_B'] = df['Product_Category_B'].astype('category')
df['Hemisferio'] = df['Hemisferio'].astype('category')
df['Estacion'] = df['Estacion'].astype('category')
df['Month'] = df['Month'].astype('category')

In [97]:
import category_encoders as ce
cat_features = ['Territory','Account_Name',\
                            'Account_Type', 'Opportunity_Type', 'Quote_Type','Delivery_Quarter', \
                            'Delivery_Terms','Last_Modified_By']
target_enc = ce.TargetEncoder(cols=cat_features)
target_enc.fit(train[cat_features], train['Stage'])

#train = train.join(target_enc.transform(train[cat_features]).add_suffix('_t'))
#test = test.join(target_enc.transform(test[cat_features]).add_suffix('_t'))
df = df.join(target_enc.transform(df[cat_features]).add_suffix('_t'))

In [ ]:
train =train[['ID']]
train = pd.merge(train,df, on= 'ID', how = 'inner') 
test =test[['ID']]
test = pd.merge(test,df, on= 'ID', how = 'inner') 
train.loc[train['Stage']=='Closed Won','Stage']=1
train.loc[train['Stage']=='Closed Lost','Stage']=0
train.Stage=train.Stage.astype('float')
train.to_csv("train_Target_enc.csv", index=False)
test.to_csv("test_Target_enc.csv", index=False)

* CatBoost Encoding

In [221]:
cat_features = ['Territory','Account_Owner','Account_Name',\
                            'Opportunity_Owner','Account_Type', 'Opportunity_Type', 'Quote_Type','Delivery_Quarter', \
                            'Delivery_Terms','Last_Modified_By',\
                            'Product_Family']
target_enc = ce.CatBoostEncoder(cols=cat_features)
target_enc.fit(train[cat_features], train['Stage'])

#train = train.join(target_enc.transform(train[cat_features]).add_suffix('_cb'))
#test = test.join(target_enc.transform(test[cat_features]).add_suffix('_cb'))
df = df.join(target_enc.transform(df[cat_features]).add_suffix('_cb'))

In [122]:
train =train[['ID']]
train = pd.merge(train,df, on= 'ID', how = 'inner') 
test =test[['ID']]
test = pd.merge(test,df, on= 'ID', how = 'inner') 
train.loc[train['Stage']=='Closed Won','Stage']=1
train.loc[train['Stage']=='Closed Lost','Stage']=0
train.Stage=train.Stage.astype('float')
train.to_csv("train_CatBoost_enc.csv", index=False)
test.to_csv("test_CatBoost_enc.csv", index=False)